In [4]:
# 데이터 형식 뜯어보기
import numpy as np
import os
import os.path
import IPython.display as ipd
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

data_path = '/content/drive/MyDrive/speech_wav_8000.npz'
speech_data = np.load(data_path)

print("checked")
print("Wave data shape : ", speech_data["wav_vals"].shape)
print("Label data shape : ", speech_data["label_vals"].shape)
print('checked2')

#데이터를 확인해보자
#데이터선택
rand = random.randint(0, len(speech_data['wav_vals']))
print("rand num : ", rand)

sr = 8000 # 1초동안 재생되는 샘플의 갯수
data = speech_data['wav_vals'][rand]
print("Wave data shape : ", data.shape)
print('label : ', speech_data['label_vals'])

ipd.Audio(data, rate = sr)

# Train/Test 데이터셋 구성하기

# Label data처리 현재는 Test형테이기 때문에 학습을 위해서는 다른 형태로 바꾸어주어야 한다

# 구분해야할 label 목록
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
label_value = target_list

# 여기에 unknown 이랑 silence를 추가해주자
label_value.append('unknown')
label_value.append('silence')
print('LABEL : ', label_value)

new_label_value=dict()
for i, l in enumerate(label_value) :
    new_label_value[l] = i
label_vlaue = new_label_value
print('Indexde LABEL :', new_label_value)

# 바꾼 숫자형으로 label을 다시 표시해보자
temp = []
for v in speech_data['label_vals']:
    temp.append(label_vlaue[v[0]])
label_data = np.array(temp)
print('label_data :',label_data)

sr = 8000
train_wav, test_wav, train_label, test_label = train_test_split(speech_data['wav_vals'],
                                                                label_data,
                                                                test_size = 0.1,
                                                                shuffle = True)
print(train_wav)

train_wav = train_wav.reshape([-1, sr, 1]) # add channel for CNN
test_wav = test_wav.reshape([-1, sr, 1])
print("Checked")

#나누어진 데이터 셋을 확인해보자
print("train data : ", train_wav.shape)
print("train labels : ", train_label.shape)
print('test data : ', test_wav.shape)
print('test labels : ', test_label.shape)

#Hyper-parameters setting
batch_size = 32
max_epochs = 10

# save point setting
checkpoint_dir = './models/wav'

checkpoint_dir

## data setting

#### 전처리랑 배치처리등을 쉽게 할 수 있게 해준다.

# tf.data.Dataset을 이용해서 데이터셋을 구성한다. 

def one_hot_label(wav, label) :
    label = tf.one_hot(label, depth = 12)
    return wav, label
print('Checked')

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_wav, train_label))
train_dataset = train_dataset.map(one_hot_label)
train_dataset = train_dataset.repeat().batch(batch_size=batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_wav, test_label))
test_dataset = test_dataset.map(one_hot_label)
test_dataset = test_dataset.batch(batch_size = batch_size)
print(test_dataset)
print("checked")

# wave classificateion 모델 구현
# Audio 데이터는 1차원데이터라서 모델을 여기에 맞추어야 한다.
from tensorflow.keras import layers
input_tensor = layers.Input(shape=(sr, 1))

x = layers.Conv1D(32, 9, padding = 'same', activation = 'relu')(input_tensor)
x = layers.Conv1D(32, 9, padding = 'same', activation = 'relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(64, 9, padding = 'same', activation = 'relu')(x)
x = layers.Conv1D(64, 9, padding = 'same', activation = 'relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(128, 9, padding = 'same', activation = 'relu')(x)
x = layers.Conv1D(128, 9, padding = 'same', activation = 'relu')(x)
x = layers.Conv1D(128, 9, padding = 'same', activation = 'relu')(x)
x = layers.MaxPool1D()(x)

x = layers.Conv1D(256, 9, padding = 'same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding = 'same', activation='relu')(x)
x = layers.Conv1D(256, 9, padding = 'same', activation='relu')(x)
x = layers.MaxPool1D()(x)
x = layers.Dropout(0.3)(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

output_tensor = layers.Dense(12)(x)
model_wav = tf.keras.Model(input_tensor, output_tensor)
model_wav.summary()

## Loss

# Loss를 정하자
optimizer = tf.keras.optimizers.Adam(1e-4)
model_wav.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  optimizer = optimizer, 
                  metrics = ['accuracy'])
print('checked')

## Training

### callback
 ###-중간중간에 원하는 동작을 진행할 수 있다

# 가중치를 저장하는 함수를 사용해보자
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir, 
                                                 save_weights_only = True, 
                                                 monitor = 'val_loss', 
                                                 mode = 'auto',
                                                 save_best_only = True,
                                                 verbose = 1)
print('checked')

#학습을 해보자!!! 
history_wav = model_wav.fit(train_dataset, epochs = max_epochs,
                            steps_per_epoch = len(train_wav) // batch_size,
                            validation_data = test_dataset,
                            validation_steps = len(test_wav) // batch_size,
                            callbacks = [cp_callback])
print("checked")

checked
Wave data shape :  (50620, 8000)
Label data shape :  (50620, 1)
checked2
rand num :  18908
Wave data shape :  (8000,)
label :  [['down']
 ['down']
 ['down']
 ...
 ['silence']
 ['silence']
 ['silence']]
LABEL :  ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']
Indexde LABEL : {'yes': 0, 'no': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5, 'on': 6, 'off': 7, 'stop': 8, 'go': 9, 'unknown': 10, 'silence': 11}
label_data : [ 3  3  3 ... 11 11 11]
[[ 1.9219516e-01  1.0275782e-01  9.1750696e-02 ...  3.4696558e-01
   4.6313461e-02  2.4940994e-01]
 [ 2.5416496e-03  5.2761468e-03  6.3464901e-04 ...  8.6495513e-04
   6.6782744e-04 -2.8720847e-03]
 [ 8.9090858e-03  6.3627690e-02  7.1682297e-02 ...  6.5097380e-03
   3.4931451e-03 -4.4511831e-03]
 ...
 [-7.5679825e-04 -1.5415990e-03 -1.7903623e-03 ...  3.2977222e-03
   2.7039545e-03  2.1060244e-03]
 [-1.1389337e-03 -3.3816700e-03 -7.6431269e-04 ... -1.1902838e-03
  -1.6585733e-03 -1.5672843e-03]
 [

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
